In [2]:
##########################################
############                  ############
############    MUST HAVES    ############
############                  ############
##########################################

print("Must Haves")


Must Haves


In [3]:
# Read CSV
# This will read in the CSVs that come from the alpha vantage website, where each CSV
# is from a separate stock symbol
import csv  

def open_csv(path_to_csv):
    data = []
    with open(path_to_csv) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            if row[0] != '':
                data.append(row)
    return data


In [4]:
# Access API
from myapikey import APIkey
from alpha_vantage.timeseries import TimeSeries


In [5]:
# must provide an api key (they are free from the alpha vantage website
# this determines how much data you can get in how much time.
def get_intraday(stock):
    ts = TimeSeries(key=APIkey, output_format='pandas', indexing_type='integer')
    return ts.get_intraday(symbol=stock, interval='1min', outputsize='full')

# provide an api key to access the data
def get_daily(stock):
    ts = TimeSeries(key=APIkey, output_format='pandas')
    data, meta_data = ts.get_daily(symbol=stock, outputsize='full')
    return data, meta_data


In [6]:
# Select stocks based on ticker symbol
stock_names = ["AAOI", "AAP", "AAPL", "ABMD", "ABT", "ACET", "ADIL", "AETI", "AFL", "AKAM", "ALGN", "ALQA", "AMD", 
"AMG", "AMRH", "AMZN", "ANY", "APVO", "AVGR", "AXSM", "BAC", "BIIB", "BLIN", "BLK", "BLRX", "BSQR", "BSX", "BTAI", 
"CGA", "CGC", "CLIR", "CMCSA", "CMG", "COP", "COTY", "COUP", "CPSH", "CSCO", "CTK", "CTRV", "CVX", "D", "DIS", 
"DUK", "EXC", "EYES", "FARO", "FB", "FCEL", "FDS", "FLKS", "FRAN", "FTNT", "GE", "GM", "GNMX", "GOOGL", "HMC", "HOV", 
"HSGX", "IGLD", "INOD", "IVZ", "JAGX", "JNJ", "JPM", "JT", "KEYS", "KOSS", "LB", "LLY", "LPTH", "M", "MDT", "MSFT", 
"MYSZ", "NDAQ", "NEE", "NFLX", "NKE", "NRG", "NVDA", "NVFY", "ORLY", "OSS", "OXBR", "PFE", "PIR", "PRGO", "QBAK", 
"QUIK", "RAD", "RBCN","RHT", "RL", "SBUX", "SESN", "SFUN", "SGMA", "SMAR", "SO", "STZ", "SYK", "T", "TNK", "TOPS", 
"TRHC", "TRIL", "TRIP", "TRNX", "TSLA", "TSN", "TTWO", "TWMC", "TWTR", "UA", "UAA", "UPL", "USB", "UXIN", "V", 
"VRSN", "VTVT", "VZ", "WDC", "WFC", "WFT", "WMT", "XOM", "ZTS"]

# at least one of these is not a real stock recognized by alpha vantage.
# I must error check these stocks.
print(len(stock_names))


130


In [7]:
# Store Stock information as a data set
stock_data = batch_get_daily(stock_names)


In [15]:
# print(stock_data[0][0])
import pickle

f = open('store_stocks.pckl', 'wb')
pickle.dump(stock_data, f)
f.close()


In [ ]:
f = open('store_stocks.pckl', 'rb')
stock_data = pickle.load(f)
f.close()


In [61]:
# Format dataset with classification (up/down)
import numpy as np

def add_classification(stocks):
    # what format of data should this function accept?
    # should I clean up csv imports vs. direct access data?
    # for each row, take the final stock price of the lower row
    #   and compare it with the current day's final stock
    #   price. If this row is lower, then "down". If this row
    #   is higher, then "up".
    
    # up = 1, down = 0
    
    # column 4 is the end of day price
    last_price = stocks[len(stocks) - 1][4]
    # the very last price will always be up. Perhaps
    # this makes my AI optimistic?
    np.insert(stocks[len(stocks) - 1], 5, 1) # up
    # stocks[len(stocks) - 1].append(1) # up
    for s in reversed(list(range(len(stocks) - 1))):
        if (stocks[s][4] - last_price) > 0:
            np.insert(stocks[s], 5, 1) # up
            # stocks[s].append(1) # up
        else:
            np.insert(stocks[s], 5, 0) # down
            # stocks[s].append(0) # down
        last_price = stocks[s][4]
    return stocks


In [65]:
# print(np.append(stock_data[0][0].to_numpy()[0], 1))
# print(np.insert(stock_data[0][0].to_numpy()[0], 5, 1))
# print(stock_data[0][0].to_numpy()[0])
print(stock_data[0][0].to_numpy())


[[1.00000e+01 1.00900e+01 9.37000e+00 9.96000e+00 9.46000e+05]
 [1.04400e+01 1.04400e+01 1.00000e+01 1.01000e+01 2.53300e+05]
 [1.00000e+01 1.01800e+01 9.71000e+00 1.00000e+01 8.49000e+04]
 ...
 [1.02500e+01 1.04300e+01 9.99000e+00 1.00100e+01 6.01777e+05]
 [9.98000e+00 1.01000e+01 9.70000e+00 9.71000e+00 6.14007e+05]
 [9.57000e+00 9.62390e+00 9.16000e+00 9.17000e+00 9.97472e+05]]


In [9]:
# (3 must-haves in one cell)
# Train
# Hidden Nodes
# Dropout
import tensorflow as tf

# example training
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)


NameError: name 'x_train' is not defined

In [ ]:
# Test
model.evaluate(x_test, y_test)


In [ ]:
# Real Life Prediction


In [ ]:
# Print results of prediction


In [ ]:
##########################################
#######                           ########
#######    WOULD LIKE TO HAVES    ########
#######                           ########
##########################################

print("would like to haves")


In [ ]:
# Save data as CSV



In [ ]:
# Wait for API (free) access limit
import time

def batch_get_daily(stock_list):
    count = 0
    data = []
    for stock in stock_list:
        if count == 5:
            time.sleep(65) # 65 seconds just in case the timing on the server or here isn't perfect
            count = 0
        data.append(get_daily(stock))
        count += 1
    return data

# for 130 stocks this should take at least half an hour


In [ ]:
# Save results of Training
# https://www.tensorflow.org/guide/keras#entire_model
model.save_weights('verdurouMKI.h5')


In [ ]:
# Load Results from Previous Training
model = tf.keras.models.load_model('verdurouMKI.h5')


In [22]:
# Scrape HTML
